<a href="https://colab.research.google.com/github/hamzapervez062/Bakery-Shop_Project_Csharp/blob/master/t5_herarchical_(2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
 #installation of libraries
 !pip install -q transformers datasets
 !pip install -q evaluate
 !pip install -q rouge_score
 !pip install -q bert-score
 !pip install -q contractions

In [ ]:
import json
import torch
from tqdm import tqdm
import pandas as pd
import re
import contractions
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM


In [ ]:
# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
device

device(type='cuda')

In [ ]:
# Load your dataset
with open("/content/regusum_dataset_segmented_final.json", "r", encoding="utf-8") as f:
    data = json.load(f)

In [ ]:
original_df = pd.DataFrame(data)
print(f"Loaded {len(original_df)} samples.")

Loaded 354 samples.


In [ ]:
df = original_df.copy()
df.head()

,ID,gold_summary,Agency ID,Title,Topics,original_content,Sections
0,IRS-2008-0041-0003,This document contains corrections to Treasury...,IRS,"Federal Register, Volume 85 Issue 113 (Thursda...","Income taxes, Reporting and recordkeeping requ...",Background\n\n The final regulations (TD 93...,"[{'header': 'Introduction', 'text': 'Backgroun..."
1,IRS-2008-0053-0009,This document contains a correction to Treasur...,IRS,"Federal Register, Volume 88 Issue 146 (Tuesday...","Income taxes, Reporting and recordkeeping requ...",Background\n\n The final regulations (TD 95...,"[{'header': 'Introduction', 'text': 'Backgroun..."
2,IRS-2008-0092-0007,This document contains a correction to a Treas...,IRS,"Federal Register, Volume 85 Issue 162 (Thursda...","Income taxes, Reporting and recordkeeping requ...",Background\n\n The final regulations (TD 96...,"[{'header': 'Introduction', 'text': 'Backgroun..."
3,IRS-2011-0050-0006,This document contains corrections to Treasury...,IRS,"Federal Register, Volume 85 Issue 46 (Monday, ...","Income taxes, Reporting and recordkeeping requ...",Background\n\n The final regulations (TD 96...,"[{'header': 'Introduction', 'text': 'Backgroun..."
4,IRS-2014-0001-0014,This document contains corrections to Treasury...,IRS,"Federal Register, Volume 85 Issue 88 (Wednesda...","Income taxes, Reporting and recordkeeping requ...",Background\n\n The final regulations (TD 98...,"[{'header': 'Introduction', 'text': 'Backgroun..."


In [ ]:
# Keep only rows where 'text' is not empty or NaN
df = df[df['original_content'] != ''].dropna(subset=['original_content'])
df.shape

(353, 7)

In [ ]:
#Keep only required columns and rename
df = df[["ID", "original_content", "gold_summary"]]

In [ ]:
#to lowercase the all text
def lowercase_text(text):
    return text.lower()

In [ ]:
def remove_url(text):
    pattern = re.compile(r'https?://\S+|www\.\S+')
    return pattern.sub(r'',text)

In [ ]:
#combinations of words that are shortened like i'm --> i am
def expand_contractions(text):
    expanded_text = contractions.fix(text)
    return expanded_text

In [ ]:
# Clean newlines in text
def clean_text(text):
    return text.replace("\n", " ").strip()

In [ ]:
#cleaning dots
def dot_cleanings(text):
  cleaned_text = re.sub(r'\.{3,}', ' ', text)
  cleaned_text = re.sub(r'\s{2,}', ' ', cleaned_text).strip()
  return cleaned_text


In [ ]:
#remove special characters like '', ', `, /, \ from text.
def cleaned_text(text):
    text = re.sub(r"[\'\"`/\\]", "", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

In [ ]:
#removing text like page no 2334
df["original_content"] = df["original_content"].str.replace(r'\n\n\[\[Page \d+\]\]\n\n', ' ', regex=True)
# removing -- and **
df["original_content"] = df["original_content"].str.replace(r"\s*\*\s*", "", regex=True).str.strip()
df["original_content"] = df["original_content"].str.replace(r'-{2,}', '', regex=True).str.strip()

In [ ]:
#apply all functions on original_content column
df.original_content = df.original_content.apply(lowercase_text)
df.original_content = df.original_content.apply(remove_url)
df.original_content = df.original_content.apply(expand_contractions)
df.original_content = df.original_content.apply(clean_text)
df.original_content = df.original_content.apply(dot_cleanings)

In [ ]:
df.gold_summary = df.gold_summary.apply(lowercase_text)
df.gold_summary = df.gold_summary.apply(remove_url)
df.gold_summary = df.gold_summary.apply(expand_contractions)
df.gold_summary = df.gold_summary.apply(clean_text)

In [ ]:
df["original_content"] = df["original_content"].str.replace(r"/", "", regex=True).str.strip()

In [ ]:
df.head()

,ID,original_content,gold_summary
0,IRS-2008-0041-0003,background the final regulations (td 9394) tha...,this document contains corrections to treasury...
1,IRS-2008-0053-0009,background the final regulations (td 9515) tha...,this document contains a correction to treasur...
2,IRS-2008-0092-0007,background the final regulations (td 9614) tha...,this document contains a correction to a treas...
3,IRS-2011-0050-0006,background the final regulations (td 9630) tha...,this document contains corrections to treasury...
4,IRS-2014-0001-0014,background the final regulations (td 9806) tha...,this document contains corrections to treasury...


In [ ]:
model_namet5 = "t5-large"
t5_tokenizer = AutoTokenizer.from_pretrained(model_namet5)
t5_model= AutoModelForSeq2SeqLM.from_pretrained(model_namet5).to(device)


In [ ]:
# Function to get token length of original_text
def get_token_length(text):
    tokens = t5_tokenizer.tokenize(text)
    return len(tokens)

In [ ]:
# Finding ids where token length of original_text exceeds 20000
exceeding_ids = []
leng = []
for idx, row in df.iterrows():
    length = get_token_length(row['original_content'])
    leng.append(length)

    if length > 20000:
        exceeding_ids.append(row['ID'])
        print(f"ID {row['ID']} has {length} tokens.")

# print("IDs with text longer than 20000 tokens:", exceeding_ids)
# print(leng)

ID IRS-2015-0030-0227 has 57638 tokens.
ID IRS-2016-0007-0008 has 23174 tokens.
ID IRS-2016-0010-0035 has 51567 tokens.
ID IRS-2016-0040-0003 has 30306 tokens.
ID IRS-2016-0047-0004 has 45942 tokens.
ID IRS-2017-0002-0013 has 23355 tokens.
ID IRS-2017-0005-0005 has 34654 tokens.
ID IRS-2018-0011-0036 has 39050 tokens.
ID IRS-2018-0027-0009 has 31439 tokens.
ID IRS-2019-0003-0004 has 22547 tokens.
ID IRS-2019-0004-0125 has 273511 tokens.
ID IRS-2019-0005-0013 has 103738 tokens.
ID IRS-2019-0012-0052 has 136028 tokens.
ID IRS-2019-0021-0012 has 42677 tokens.
ID IRS-2019-0022-0142 has 241024 tokens.
ID IRS-2019-0023-0027 has 64172 tokens.
ID IRS-2019-0027-0022 has 53230 tokens.
ID IRS-2019-0028-0011 has 86098 tokens.
ID IRS-2019-0029-0034 has 55804 tokens.
ID IRS-2019-0033-0020 has 57414 tokens.
ID IRS-2019-0034-0028 has 108986 tokens.
ID IRS-2019-0038-0018 has 88505 tokens.
ID IRS-2019-0041-0008 has 63841 tokens.
ID IRS-2019-0044-0010 has 27957 tokens.
ID IRS-2019-0049-0019 has 25654 tok

In [ ]:
# Calculate token lengths for all rows (original_content) and store them in a new column
df['token_length'] = df['original_content'].apply(get_token_length)

# Get IDs where token length > 20000
exceeding_ids = df.loc[df['token_length'] > 20000, 'ID'].tolist()

print("IDs with text longer than 20000 tokens:", exceeding_ids)

# Create a new DataFrame with rows where token length ≤ 20000
df_less_than_20000 = df[df['token_length'] <= 20000].copy()

print(f"Rows with ≤20000 tokens: {len(df_less_than_20000)}")


IDs with text longer than 20000 tokens: ['IRS-2015-0030-0227', 'IRS-2016-0007-0008', 'IRS-2016-0010-0035', 'IRS-2016-0040-0003', 'IRS-2016-0047-0004', 'IRS-2017-0002-0013', 'IRS-2017-0005-0005', 'IRS-2018-0011-0036', 'IRS-2018-0027-0009', 'IRS-2019-0003-0004', 'IRS-2019-0004-0125', 'IRS-2019-0005-0013', 'IRS-2019-0012-0052', 'IRS-2019-0021-0012', 'IRS-2019-0022-0142', 'IRS-2019-0023-0027', 'IRS-2019-0027-0022', 'IRS-2019-0028-0011', 'IRS-2019-0029-0034', 'IRS-2019-0033-0020', 'IRS-2019-0034-0028', 'IRS-2019-0038-0018', 'IRS-2019-0041-0008', 'IRS-2019-0044-0010', 'IRS-2019-0049-0019', 'IRS-2019-0050-0057', 'IRS-2019-0054-0015', 'IRS-2019-0057-0017', 'IRS-2019-0059-0008', 'IRS-2020-0001-0009', 'IRS-2020-0002-0018', 'IRS-2020-0006-0021', 'IRS-2020-0008-0032', 'IRS-2020-0013-0091', 'IRS-2020-0017-0017', 'IRS-2020-0018-0023', 'IRS-2020-0019-0017', 'IRS-2020-0020-0011', 'IRS-2020-0023-0002', 'IRS-2020-0026-0005', 'IRS-2020-0028-0013', 'IRS-2020-0030-0020', 'IRS-2020-0033-0003', 'IRS-2020-003

In [ ]:
#length of those docs that are exceeding 20000 tokens' length
len(exceeding_ids )

154

In [ ]:
# Function to get token length of gold_summary
def get_token_length_gold_summary(gold_summary):
    tokens = t5_tokenizer.tokenize(gold_summary)
    return len(tokens)

In [ ]:
# Calculate token lengths for all rows and store in a new column
df_less_than_20000['token_gold_summary'] = df_less_than_20000['gold_summary'].apply(get_token_length_gold_summary)

In [ ]:
df_less_than_20000.head(10)

,ID,original_content,gold_summary,token_length,token_gold_summary
0,IRS-2008-0041-0003,background the final regulations (td 9394) tha...,this document contains corrections to treasury...,781,126
1,IRS-2008-0053-0009,background the final regulations (td 9515) tha...,this document contains a correction to treasur...,700,155
2,IRS-2008-0092-0007,background the final regulations (td 9614) tha...,this document contains a correction to a treas...,1104,124
3,IRS-2011-0050-0006,background the final regulations (td 9630) tha...,this document contains corrections to treasury...,217,97
4,IRS-2014-0001-0014,background the final regulations (td 9806) tha...,this document contains corrections to treasury...,269,132
5,IRS-2014-0030-0004,authority this document amends the income tax ...,this document contains final regulations relat...,14034,91
8,IRS-2016-0007-0009,background the final regulations (td 9981) tha...,this document contains a correction to treasur...,208,120
9,IRS-2016-0007-0010,background the final regulations (td 9981) tha...,this document corrects the correction to treas...,171,122
11,IRS-2016-0038-0007,background this document contains amendments t...,this document contains the final regulations t...,8973,74
13,IRS-2016-0044-0011,background section 401(a)(11) of the internal ...,this document sets forth final regulations pro...,17591,136


**Chunking Method Implementation with dynamic technique**

In [ ]:
#creating chunks """Splits text into chunks based on T5 token count."""

def split_text_t5(text, chunk_size=500, chunk_overlap=200):
    token_ids = t5_tokenizer.encode(text, add_special_tokens=False)
    chunks = []
    start = 0
    while start < len(token_ids):
        end = min(start + chunk_size, len(token_ids))

        chunk_ids = token_ids[start:end]
        chunk_text = t5_tokenizer.decode(chunk_ids , skip_special_tokens=True)

        chunks.append(chunk_text)

        # If the next start would be beyond the last new token, break
        if end == len(token_ids):
            break

        start += chunk_size - chunk_overlap # move window forward

    return chunks

In [ ]:
df_less_than_20000.tail(5)

,ID,original_content,gold_summary,token_length,token_gold_summary
345,SEC-2024-1184-0001,the commission is adopting new 17 cfr 270.3c-7...,the securities and exchange commission (``comm...,6822,146
348,SEC-2024-1493-0001,"we are adopting an updated filer manual, volum...",the securities and exchange commission (``comm...,2170,129
349,SEC-2024-1627-0001,the commission is amending the following rules...,the securities and exchange commission (``comm...,18893,102
351,SEC-2024-1650-0001,the commission is adopting technical amendment...,the securities and exchange commission (``comm...,1432,293
352,SEC-2024-1821-0001,i. background the commission is amending the i...,the securities and exchange commission (``sec'...,1938,55


In [ ]:
final_chunks = {}
generated_summaries = []

# Step 1: Chunk the documents using T5 tokenization
for _, row in tqdm(df_less_than_20000.iterrows(), total=len(df_less_than_20000)):

    input_text = row['original_content']
    gold_summary = row['gold_summary']  # The gold summary text
    doc_id = row["ID"]

    # Count tokens using T5 tokenizer
    total_tokens_input = len(t5_tokenizer.encode(input_text, add_special_tokens=False))
    gold_summary_tokens = len(t5_tokenizer.encode(gold_summary, add_special_tokens=False))

    #for shorter chunks < 512
    if total_tokens_input < 512:
        final_chunks[doc_id] = {
            "chunks": [input_text],
            "gold_length": gold_summary_tokens
        }

    #for longer documents chunking
    else:

        split_chunks = split_text_t5(input_text, chunk_size=500, chunk_overlap=200) # calling function (split_text_t5)
        print(f"Splitting ID {doc_id} into {len(split_chunks)} chunks (T5-based)")
        final_chunks[doc_id] = {
            "chunks": split_chunks,
            "gold_length": gold_summary_tokens
        }

# Step 2: Summarize each chunk with proportional length allocation
for doc_id, data in tqdm(final_chunks.items()):
    doc_chunks = data["chunks"]
    gold_length = data["gold_length"]  # T5 token length of gold summary
    num_chunks = len(doc_chunks)

    # Allocate token budget proportionally to each chunk
    chunk_target_len = max(15, gold_length // num_chunks)

    print(f"Processing document with ID: {doc_id} - {num_chunks} chunks ")

    predicted_chunks = []
    for chunk in doc_chunks:
      #tokenizing
        input_ids = t5_tokenizer(
            "summarize: " + chunk,
            return_tensors="pt",
            truncation=False,
            max_length=512 # T5 limited to 512 tokens
        ).input_ids.to("cuda")

        #summary generation
        output_ids = t5_model.generate(
            input_ids,
            max_length=chunk_target_len,
        )
        #decode into text
        summary = t5_tokenizer.decode(output_ids[0], skip_special_tokens=True)
        predicted_chunks.append(summary)

    # Concatenate chunk summaries for final prediction
    predicted_summary = " ".join(predicted_chunks)
    generated_summaries.append(predicted_summary)


  3%|▎         | 6/199 [00:00<00:03, 50.65it/s]

Splitting ID IRS-2008-0041-0003 into 2 chunks (T5-based)
Splitting ID IRS-2008-0053-0009 into 2 chunks (T5-based)
Splitting ID IRS-2008-0092-0007 into 4 chunks (T5-based)
Splitting ID IRS-2014-0030-0004 into 47 chunks (T5-based)
Splitting ID IRS-2016-0038-0007 into 30 chunks (T5-based)


  9%|▊         | 17/199 [00:00<00:04, 40.14it/s]

Splitting ID IRS-2016-0044-0011 into 58 chunks (T5-based)
Splitting ID IRS-2016-0054-0015 into 25 chunks (T5-based)
Splitting ID IRS-2017-0005-0007 into 2 chunks (T5-based)
Splitting ID IRS-2017-0010-0003 into 23 chunks (T5-based)
Splitting ID IRS-2017-0017-0003 into 5 chunks (T5-based)
Splitting ID IRS-2018-0008-0019 into 8 chunks (T5-based)
Splitting ID IRS-2018-0011-0037 into 3 chunks (T5-based)
Splitting ID IRS-2018-0040-0051 into 2 chunks (T5-based)


 16%|█▌        | 32/199 [00:00<00:03, 43.46it/s]

...

 99%|█████████▉| 197/199 [29:44<00:30, 15.47s/it]

Processing document with ID: SEC-2024-1650-0001 - 5 chunks 


 99%|█████████▉| 198/199 [29:53<00:13, 13.54s/it]

Processing document with ID: SEC-2024-1821-0001 - 6 chunks 


100%|██████████| 199/199 [29:56<00:00,  9.03s/it]


In [ ]:
len(generated_summaries)

199

In [ ]:
df_less_than_20000.head(10)

,ID,original_content,gold_summary,token_length,token_gold_summary
0,IRS-2008-0041-0003,background the final regulations (td 9394) tha...,this document contains corrections to treasury...,781,126
1,IRS-2008-0053-0009,background the final regulations (td 9515) tha...,this document contains a correction to treasur...,700,155
2,IRS-2008-0092-0007,background the final regulations (td 9614) tha...,this document contains a correction to a treas...,1104,124
3,IRS-2011-0050-0006,background the final regulations (td 9630) tha...,this document contains corrections to treasury...,217,97
4,IRS-2014-0001-0014,background the final regulations (td 9806) tha...,this document contains corrections to treasury...,269,132
5,IRS-2014-0030-0004,authority this document amends the income tax ...,this document contains final regulations relat...,14034,91
8,IRS-2016-0007-0009,background the final regulations (td 9981) tha...,this document contains a correction to treasur...,208,120
9,IRS-2016-0007-0010,background the final regulations (td 9981) tha...,this document corrects the correction to treas...,171,122
11,IRS-2016-0038-0007,background this document contains amendments t...,this document contains the final regulations t...,8973,74
13,IRS-2016-0044-0011,background section 401(a)(11) of the internal ...,this document sets forth final regulations pro...,17591,136


In [ ]:

df_less_than_20000['generated_summary'] = generated_summaries

In [ ]:

# Function to get token length of llm generated summary
def token_length_generated_summary(generated_summary):
    tokens = t5_tokenizer.tokenize(generated_summary)
    return len(tokens)
#
df_less_than_20000['tokens_generated_summary'] = df_less_than_20000['generated_summary'].apply(token_length_generated_summary)


In [ ]:
df_less_than_20000.head(10)

,ID,original_content,gold_summary,token_length,token_gold_summary,generated_summary,tokens_generated_summary
0,IRS-2008-0041-0003,background the final regulations (td 9394) tha...,this document contains corrections to treasury...,781,126,final regulations (td 9394) contained errors t...,113
1,IRS-2008-0053-0009,background the final regulations (td 9515) tha...,this document contains a correction to treasur...,700,155,treasury decision 9515 was corrected at 76 cfr...,132
2,IRS-2008-0092-0007,background the final regulations (td 9614) tha...,this document contains a correction to a treas...,1104,124,final regulations (td 9614) contained errors t...,119
3,IRS-2011-0050-0006,background the final regulations (td 9630) tha...,this document contains corrections to treasury...,217,97,final regulations (td 9630) contain an error t...,63
4,IRS-2014-0001-0014,background the final regulations (td 9806) tha...,this document contains corrections to treasury...,269,132,final regulations (td 9806) contained errors t...,57
5,IRS-2014-0030-0004,authority this document amends the income tax ...,this document contains final regulations relat...,14034,91,irs regulations amend section 752 of code rega...,651
8,IRS-2016-0007-0009,background the final regulations (td 9981) tha...,this document contains a correction to treasur...,208,120,final regulations (td 9981) that are subject o...,56
9,IRS-2016-0007-0010,background the final regulations (td 9981) tha...,this document corrects the correction to treas...,171,122,final regulations (td 9981) that are the subje...,74
11,IRS-2016-0038-0007,background this document contains amendments t...,this document contains the final regulations t...,8973,74,irs is required to charge user fees for proces...,417
13,IRS-2016-0044-0011,background section 401(a)(11) of the internal ...,this document sets forth final regulations pro...,17591,136,401(a)(11) of the internal revenue code provid...,804


In [ ]:
# Importing evaluate library
import evaluate
rouge_metric = evaluate.load("rouge")
rouge_results = rouge_metric.compute(predictions=df_less_than_20000.generated_summary, references=df_less_than_20000.gold_summary, use_stemmer=True)
print(rouge_results)

{'rouge1': np.float64(0.3181544008513224), 'rouge2': np.float64(0.10093954087935489), 'rougeL': np.float64(0.18038433429135386), 'rougeLsum': np.float64(0.1803582756948826)}


In [ ]:
# BERTScore calculation
predictions = df_less_than_20000["generated_summary"].tolist()
references = df_less_than_20000["gold_summary"].tolist()

bertscore_metric = evaluate.load('bertscore')
bert = bertscore_metric.compute(predictions=predictions, references=references, lang="en")

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
sum(bert['f1']) / len(bert['f1'])

0.8182603072880501

**Herarchical Method Implementation**

In [ ]:
#
herarchical_summaries = []

for _, row in tqdm(df_less_than_20000.iterrows(), total=len(df_less_than_20000)):
    input_text = row['generated_summary'] #getting chunk based generated summaries
    target_length = int(row['token_gold_summary'])  # fetching every rows gold_summary tokens' length for dynamic length

    # Prepare input
    #tokenizing
    input_ids = t5_tokenizer(
        "summarize: " + input_text,
        return_tensors="pt",

        max_length=512 #because t5 limit to 512 tokens
    ).input_ids.to("cuda")

    # Generate with fixed length
    output_ids = t5_model.generate(
        input_ids,
        max_length=target_length, #dynamica max_length generated for each column
    )
    #decode to text back
    summary = t5_tokenizer.decode(output_ids[0], skip_special_tokens=True) # generating summaries
    herarchical_summaries.append(summary)  #append summaries to list




100%|██████████| 199/199 [07:28<00:00,  2.25s/it]


In [ ]:
len(herarchical_summaries)

199

In [ ]:
df_less_than_20000['her_generated_summary'] = herarchical_summaries #putting new summaries in new column

In [ ]:
# Importing evaluate library
import evaluate
rouge_metric = evaluate.load("rouge")
rouge_results = rouge_metric.compute(predictions=df_less_than_20000.her_generated_summary, references=df_less_than_20000.gold_summary, use_stemmer=True)
print(rouge_results)


{'rouge1': np.float64(0.2520255670268635), 'rouge2': np.float64(0.0764821573448203), 'rougeL': np.float64(0.1620459789418949), 'rougeLsum': np.float64(0.1615695074446009)}


In [ ]:
# BERTScore calculation
predictions = df_less_than_20000["her_generated_summary"].tolist()
references = df_less_than_20000["gold_summary"].tolist()

bertscore_metric = evaluate.load('bertscore')
bert = bertscore_metric.compute(predictions=predictions, references=references, lang="en")

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
bertt = sum(bert['f1']) / len(bert['f1'])

In [ ]:
bertt

0.8191639661189899

In [ ]:
# 0.8049128425121307

In [ ]:

# Prepare dictionary with metric names as keys and scores as values
rouge_dict = {
    "Model": "T5 (Pre-Trained)",
    "Experiment": "Herarchical",
    "ROUGE-1": rouge_results["rouge1"],
    "ROUGE-2": rouge_results["rouge2"],
    "ROUGE-L": rouge_results["rougeL"],
    "Bart ":  bertt,
}

# Create DataFrame with one row
rouge_df = pd.DataFrame([rouge_dict])

rouge_df


,Model,Experiment,ROUGE-1,ROUGE-2,ROUGE-L,Bart
0,T5 (Pre-Trained),Herarchical,0.252997,0.076905,0.162117,0.819309
